In [25]:
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interactive_output, Layout
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from cartopy import crs, feature

from utils import load_teseo_particles_txt

items_layout = Layout( width='auto')


In [26]:
directory = '/home/aragong/repositories/plotting-notebooks/mock/TESEO_TOOLS_459612f0'
pattern = '*particles*.txt'
grid = '/home/aragong/repositories/plotting-notebooks/mock/TESEO_TOOLS_459612f0/input/grid.dat'

df = load_teseo_particles_txt(pattern, directory)


In [27]:
play = widgets.Play(
    value=df.time.unique().min(),
    min=df.time.unique().min(),
    max=df.time.unique().max(),
    step=np.unique(np.diff(df.time.unique())),
    interval=500,
    description="Press play",
    disabled=False,
)

time_slider = widgets.FloatSlider(
    value=df.time.unique().min(),
    min=df.time.unique().min(),
    max=df.time.unique().max(),
    step=np.unique(np.diff(df.time.unique())),
    description='Time (h):',
    readout_format='.1f',
)
widgets.jslink((play, 'value'), (time_slider, 'value'))

dx_slider = widgets.FloatSlider(
    value=1.000,
    min=0.000,
    max=2.000,
    step=0.02,
    description='Zoom out:',
    readout_format='.3f',
)

psize_slider =  widgets.IntSlider(
    value=1,
    min=1,
    max=50,
    step=1,
    description='Particle size:',
)

grid_checkbox = widgets.Checkbox(
    value=True,
    description='Gridlines',
    disabled=False,
    indent=False
)

pmarker_box = widgets.Dropdown(
    options=[("point", "o"), ("cross", "x"), ("square", "s")],
    value="o",
    description='Marker',
)

pcolor_box = widgets.Dropdown(
    options=["blue", "red", "black", "green", "yellow", "brown"],
    value="blue",
    description='Color',
)

time_panel = widgets.HBox([play, time_slider])
control_panel = widgets.HBox([widgets.VBox([grid_checkbox, dx_slider, psize_slider]), widgets.VBox([pmarker_box, pcolor_box])])
tab = widgets.Tab()
tab.children = [widgets.VBox([grid_checkbox, dx_slider]), widgets.VBox([psize_slider, pmarker_box, pcolor_box])]
tab._titles = {0:"map properties", 1:"particle properties"}

In [28]:
def plot_teseo_particles(t, dx, pmarker, pcolor, psize,  gshhs=None, extent=None):
    
    df_tmp = df.loc[df.time == t]

    if dx is not None:
        extent = [df_tmp.lon.min()-dx, df_tmp.lon.max()+dx, df_tmp.lat.min()-dx, df_tmp.lat.max()+dx]

    fig = plt.figure(figsize=[16, 12])
    ax = fig.add_subplot(1, 1, 1, projection=crs.PlateCarree())

    if extent is not None:
        ax.set_extent(extent, crs=crs.PlateCarree())
    if gshhs:
        ax.add_feature(feature.GSHHSFeature(scale="full")) 
    
    ax.add_feature(feature.LAND)
    ax.add_feature(feature.OCEAN)
    ax.add_feature(feature.COASTLINE)
    ax.add_feature(feature.BORDERS, linestyle=":")
    ax.add_feature(feature.LAKES, alpha=0.5)
    ax.add_feature(feature.RIVERS)
    ax.scatter(df_tmp.lon, df_tmp.lat, marker=pmarker, c=pcolor, s=psize)


In [29]:
out = interactive_output(plot_teseo_particles, {"t":time_slider, "dx":dx_slider, "pmarker":pmarker_box, "pcolor": pcolor_box, "psize": psize_slider})

In [30]:
display(tab, out, time_panel)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<Figure size 1152x864 with 1 Axes>', '…